In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import random
from PIL import Image
import seaborn as sb
import torch
from torchvision import models
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models.vision_transformer import vit_b_16, ViT_B_16_Weights
from torchvision.utils import make_grid
from torch.autograd import Variable
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support
import shutil  ##offers high-level operation on a file like a copy, create, and remote operation on the file
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
if not os.path.exists("working"):
    os.mkdir("working")
code_dir = "./working/code"
model_dir = "./working/model"
output_dir = "./working/output"

if not os.path.exists(code_dir):
    os.mkdir(code_dir)

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [4]:
os.getcwd()

'/Users/ananyapurkait/Study Files and Folders/Semester Study/Sem VII/CS435/Project'

In [5]:
os.chdir("working/code")

In [6]:
os.getcwd()

'/Users/ananyapurkait/Study Files and Folders/Semester Study/Sem VII/CS435/Project/working/code'

In [7]:
import sys
sys.path.append("../../model")
import convnext

In [8]:
from timm.models import create_model
# def ConvNeXt_model():
#     # Instantiate ConvNeXt Tiny with pretrained weights
#     model = create_model('convnext_tiny', pretrained=True)
#     return model

#Instantiating the ConvNeXt model
def ConvNeXt_model():
    model_conv=convnext.ConvNeXt()
    #model_conv = create_model('convnext_tiny', pretrained=True)
    state_dict = torch.load('../../model/convnext_tiny_1k_224_ema.pth')
    model_conv.load_state_dict(state_dict["model"])
    
    return model_conv

def ViT_model():
    model_vit=vit_b_16(pretrained=True)
    return model_vit

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [12]:
train_acc, teste_acc, train_loss, teste_loss = [], [], [], []
train_precision, teste_precision, train_recall, teste_recall = [], [], [], []
train_f1, teste_f1 = [], []
df = pd.DataFrame(columns=['Modelo','Experimento','Epoch', 'Train ACC', 'Train Loss', 'Train F1', 'Test ACC', 'Test Loss', 'Test F1'])

In [13]:
def full_data_transform(model_type, data_fraction, batch_size):
    # Data transformations
    if model_type== 'convnext':
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    elif model_type == 'vit':
        transform = ViT_B_16_Weights.IMAGENET1K_V1.transforms()
            
            
    local_arquivos='../../newdata/real_vs_fake/real-vs-fake'
    full_train_dataset = ImageFolder(local_arquivos + "/train", transform=transform)
    full_test_dataset = ImageFolder(local_arquivos + "/test", transform=transform)
    # Determine the number of objects to be selected
    num_train_data = int(len(full_train_dataset) * data_fraction)
    num_test_data = int(len(full_test_dataset) * data_fraction)
    
    

    # Randomly select objects for the datasets
    train_indices = random.sample(range(len(full_train_dataset)), num_train_data)
    test_indices = random.sample(range(len(full_test_dataset)), num_test_data)
    # Create datasets with randomly selected objects
    train_dataset = torch.utils.data.Subset(full_train_dataset, train_indices)
    test_dataset = torch.utils.data.Subset(full_test_dataset, test_indices)
    
    # Create dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=int(batch_size/2), shuffle=False)
        
        
    return train_dataloader, test_dataloader

In [14]:
def ft_data_transform(model_type, data_fraction, batch_size):
    # Data transformations
    if model_type== 'convnext':
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    elif model_type == 'vit':
        transform = ViT_B_16_Weights.IMAGENET1K_V1.transforms()

    local_arquivos='../../newdata/real_vs_fake/real-vs-fake'
    full_test_dataset = ImageFolder(local_arquivos + "/test", transform=transform)
            
    # Determine the number of objects to be selected
    num_test_data = int(len(full_test_dataset) * data_fraction)
    # Randomly select objects for the datasets
    test_indices = random.sample(range(len(full_test_dataset)), num_test_data)

    # Create datasets with randomly selected objects
    test_dataset = torch.utils.data.Subset(full_test_dataset, test_indices)
    
    # Create dataloaders
    test_dataloader = DataLoader(test_dataset, batch_size=int(batch_size/2), shuffle=False)

    return test_dataloader

In [15]:
def AutoAugment_transform(model_type, train_indices, batch_size):
    if model_type== 'convnext':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.AutoAugment(), ## for automatically enhancing the diversity and quality of the training data
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    elif model_type == 'vit':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(224, interpolation=Image.BILINEAR),
            transforms.CenterCrop(224),
            transforms.AutoAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
        ])
    local_arquivos='../../newdata/real_vs_fake/real-vs-fake'
    augmented_train_dataset = ImageFolder(local_arquivos + "/train", transform=augmentation_transforms)
    augmented_train_dataset = torch.utils.data.Subset(augmented_train_dataset, train_indices)
    augmented_train_dataloader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True)
    
    return augmented_train_dataloader

In [16]:
def RandAugment_transform(model_type, train_indices, batch_size):
    if model_type== 'convnext':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.RandAugment(),#RandAugment simplifies the process by using a reduced search space. Instead of searching for the best policy from scratch
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    elif model_type == 'vit':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(224, interpolation=Image.BILINEAR),
            transforms.CenterCrop(224),
            transforms.RandAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
        ])
        
    local_arquivos='../../newdata/real_vs_fake/real-vs-fake'
    augmented_train_dataset = ImageFolder(local_arquivos + "/train", transform=augmentation_transforms)
    augmented_train_dataset = torch.utils.data.Subset(augmented_train_dataset, train_indices)
    augmented_train_dataloader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True)
    
    return augmented_train_dataloader
    

In [17]:
def Auto_RandAugment_transform(model_type, train_indices, batch_size):
    if model_type== 'convnext':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.AutoAugment(),
            transforms.RandAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    elif model_type == 'vit':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(224, interpolation=Image.BILINEAR),
            transforms.CenterCrop(224),
            transforms.AutoAugment(),
            transforms.RandAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
        ])
        
    local_arquivos='../../newdata/real_vs_fake/real-vs-fake'
    augmented_train_dataset = ImageFolder(local_arquivos + "/train", transform=augmentation_transforms)
    augmented_train_dataset = torch.utils.data.Subset(augmented_train_dataset, train_indices)
    augmented_train_dataloader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True)
    
    return augmented_train_dataloader

In [18]:
os.getcwd()

'/Users/ananyapurkait/Study Files and Folders/Semester Study/Sem VII/CS435/Project/working/code'

In [19]:
 # Generic training function
def train(model, dataloader, criterion, optimizer, scheduler, device, ft, epoch, exp, model_type):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    y_true,y_pred=[], []
    
    loop = tqdm(enumerate(dataloader), total=len(dataloader))
    for batch_idx, (images, labels) in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad() #Clears the gradients of all optimized tensors before performing backpropagation.
        if ft==False:    #Conditionally executes either fine-tuning or regular training based on the ft flag
            for param in model.parameters():
                param.requires_grad=False   #Freeze model parameters
                
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.step()
            for param in model.parameters():
                param.requires_grad=True
            #After the optimization step, this loop iterates over all parameters of the model again and sets requires_grad attribute to True. This unfreezes the model's weights and biases, allowing them to be updated during subsequent iterations of training.
        else:    
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        running_loss += loss.item()
        predicted = outputs.argmax(dim = 1)
    
        y_true.extend(labels.cpu().tolist())
        y_pred.extend(predicted.cpu().tolist())

        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        loop.set_description(f"[Epoch {(epoch+1)}]")
        loop.set_postfix(loss=loss.item())
        
    if scheduler:
        scheduler.step()
        
    train_loss = running_loss / len(dataloader.dataset)  
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)

    print(f"Train Loss: {train_loss:.6f} | Train Accuracy: {(accuracy * 100):.2f}% | Train F1-Score: {f1:.6f}")
    
    model_name= f'model_{model_type}_params_exp_{exp}.pth'
    torch.save(model.state_dict(), os.path.join('../../model', model_name))
   
    return train_loss, accuracy, f1

In [20]:
# Generic testing function
def test(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    y_pred, y_true= [], []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            

            running_loss += loss.item()
            predicted = outputs.argmax(dim = 1)
            
            y_true.extend(labels.cpu().tolist())
            y_pred.extend(predicted.cpu().tolist())
            
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss = running_loss / len(list(dataloader.dataset))
    test_accuracy = accuracy = accuracy_score(y_true, y_pred)
    precision, recall, test_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)
    
    print(f"Test Loss: {test_loss:.6f} | Test Accuracy: {(test_accuracy * 100):.2f}% | Test F1-Score: {test_f1:.6f}")
    return test_loss, test_accuracy, test_f1

In [21]:
#Generic function for training and testing
def train_model(model_type, exp,model, train_dataloader, test_dataloader, criterion, optimizer, scheduler, device, num_epochs, ft, num):
    model=model.to(device)
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []

    for epoch in range(num_epochs):
        print('----------------------------------------------------------------------------')
        train_loss, train_accuracy, train_f1 = train(model, train_dataloader, criterion, optimizer, scheduler, device, ft, epoch, exp, model_type)
        test_loss, test_accuracy,test_f1 = test(model, test_dataloader, criterion, device)
        val=str(num)+str(epoch+1)
        df.loc[val]=[model_type, exp, epoch+1, train_accuracy, train_loss, train_f1, test_accuracy, test_loss, test_f1]
        df.to_csv('metricas.csv', index = False)
        print('\n')
        
        
        
    return train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1

In [22]:
def run_scenario(model_type, scenario, data_fraction, num_epochs=10, batch_size=32, learning_rate=0.001):
    num_classes = 2  
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    num_train_data = int((100000) * data_fraction)
    num_test_data = int((20000) * data_fraction)
    
    train_indices = random.sample(range(65000), num_train_data)
    test_indices = random.sample(range(20000), num_test_data)
    
    print(f'Number of training objects: {num_train_data}')
    print(f'Number of test objects: {num_test_data}')
    print("============================================================================")
    
    if scenario == 1:
        
        train_dataloader, test_dataloader= full_data_transform(model_type, data_fraction, batch_size)
        
        
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=1

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=9
        else:
            raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")
        
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experimento 1 - ConvNeXt (Sem Fine-Tuning)")
        elif model_type == 'vit':
            print("Experimento 1 - ViT (Sem Fine-Tuning)")
        
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,1, model, train_dataloader, test_dataloader,
                                                                                    criterion, optimizer, scheduler, device, 1, False, num)
    elif scenario == 2:
        
        train_dataloader, test_dataloader= full_data_transform(model_type, data_fraction, batch_size)
        
       
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=2

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=10
        else:
            raise ValueError("O parâmetro 'model_type' deve ser 'convnext' ou 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
             print("Experiment 2 - ConvNeXt (With Fine-Tuning)")
        elif model_type == 'vit':
            print("Experiment 2 - ViT (With Fine-Tuning)")
        
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,2, model, train_dataloader, test_dataloader,
                                                                                    criterion, optimizer, scheduler, device, num_epochs, True, num)
    elif scenario == 3:
        
        augmented_train_dataloader = AutoAugment_transform(model_type, train_indices, batch_size)
        test_dataloader=ft_data_transform(model_type, data_fraction, batch_size)
        

        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=3

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=11
        else:
            raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experiment 3 - ConvNeXt (No Fine-Tuning with AutoAugment")
        elif model_type == 'vit':
            print("Experiment 3 - ViT (No Fine-Tuning with AutoAugment)")
        
        
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,3, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, 1, False, num)
      
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Replace num_classes with the correct number of classes
            num=4
            
        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=12
        else:
            raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experiment 4 - ConvNeXt (With Fine-Tuning and with AutoAugment)")
        elif model_type == 'vit':
            print("Experiment 4 - ViT (With Fine-Tuning and with AutoAugment)")
        
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,4, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, num_epochs, True, num)
    
    elif scenario == 4:
        augmented_train_dataloader = RandAugment_transform(model_type, train_indices, batch_size)
        test_dataloader=ft_data_transform(model_type, data_fraction, batch_size)
        
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=5
            
        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=13
        else:
            raise ValueError("O parâmetro 'model_type' deve ser 'convnext' ou 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
  
        if model_type == 'convnext':
            print("Experiment 5 - ConvNeXt (No Fine-Tuning and with RandAugment)")
        elif model_type == 'vit':
            print("Experiment 5 - ViT (No Fine-Tuning and with RandAugment)")    
        
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,5, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, 1, False, num)
        
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  #Replace num_classes with the correct number of classes
            num=6
            
        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=14
        else:
            raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experiment 6 - ConvNeXt (With Fine-Tuning and with RandAugment)")
        elif model_type == 'vit':
            print("Experiment 6 - ViT (With Fine-Tuning and with RandAugment)")
        
        
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,6, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, num_epochs, True, num) 

    elif scenario == 5:
        augmented_train_dataloader = Auto_RandAugment_transform(model_type, train_indices, batch_size)
        test_dataloader=ft_data_transform(model_type, data_fraction, batch_size)
        
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=7
            
        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=15
        else:
            raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experiment 7 - ConvNeXt (No Fine-Tuning and with RandAugment and AutoAugment)")
        elif model_type == 'vit':
            print("Experiment 7 - ViT (No Fine-Tuning and with RandAugment and AutoAugment)")
            
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,7, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, 1, False, num)
      
        # Select the model
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes) 
            num=8
            
        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=16
        else:
            raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experiment 8 - ConvNeXt (With Fine-Tuning and with RandAugment and AutoAugment)")
        elif model_type == 'vit':
            print("Experiment 8 - ViT (With Fine-Tuning and with RandAugment and AutoAugment)")

        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,8, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, num_epochs, True, num)
    else:
        raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")



In [23]:
import matplotlib.pyplot as plt

def plot_experiment_graphs(df, experiment_number):
    experiment_df = df[df['Experimento'] == experiment_number]

    #  Plot epoch x train F1
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Train F1'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Train F1')
    plt.title(f'Experiment {experiment_number} - Train F1')
    plt.tight_layout()
    plt.show()
    
    # Plot epoch x train accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Train ACC'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Train Accuracy')
    plt.title(f'Experiment {experiment_number} - Train Accuracy')
    plt.tight_layout()
    plt.show()
    
    # Plot epoch x train loss
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Train Loss'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Train Loss')
    plt.title(f'Experimento {experiment_number} - Trai loss')
    plt.tight_layout()
    plt.show()
    
    # Plot epoch x test F1
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Test F1'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Test F1')
    plt.title(f'Experiment {experiment_number} - Test F1')
    plt.tight_layout()
    plt.show()
    
    # Plot epoch x test accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Test ACC'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Test Accuracy')
    plt.title(f'Experiment {experiment_number} - Test Accuracy')
    plt.tight_layout()
    plt.show()
    
    # Plot epoch x test loss
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Test Loss'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Test loss')
    plt.title(f'Experimento {experiment_number} - Test loss')
    plt.tight_layout()
    plt.show()

def plot_general_graphs(df):
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

    # Plot epoch x train F1
    ax1 = axes[0, 0]
    for experiment_number in df['Experimento'].unique():
        experiment_df = df[df['Experimento'] == experiment_number]
        ax1.plot(experiment_df['Epoch'], experiment_df['Train F1'], marker='o', label=f'Experimento {experiment_number}')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Train F1')
    ax1.set_title('Overall - Train F1')
    ax1.legend()

    # Plot epoch x train loss
    ax2 = axes[0, 1]
    for experiment_number in df['Experiment'].unique():
        experiment_df = df[df['Experiment'] == experiment_number]
        ax2.plot(experiment_df['Epoch'], experiment_df['Train Loss'], marker='o', label=f'Experiment {experiment_number}')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Train Loss')
    ax2.set_title('Overall - Train Loss')
    ax2.legend()

    # Plot epoch x test F1
    ax3 = axes[1, 0]
    for experiment_number in df['Experimento'].unique():
        experiment_df = df[df['Experimento'] == experiment_number]
        ax3.plot(experiment_df['Epoch'], experiment_df['Test F1'], marker='o', label=f'Experimento {experiment_number}')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Test F1')
    ax3.set_title('Overall - Test F1')
    ax3.legend()

    # Plot epoch x test loss
    ax4 = axes[1, 1]
    for experiment_number in df['Experiment'].unique():
        experiment_df = df[df['Experiment'] == experiment_number]
        ax4.plot(experiment_df['Epoch'], experiment_df['Test Loss'], marker='o', label=f'Experimento {experiment_number}')
    ax4.set_xlabel('Epoch')
    ax4.set_ylabel('Test Loss')
    ax4.set_title('Overall - Test Loss')
    ax4.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
pip install h5py

In [40]:
import h5py

# After each run of the scenario, save the model to a .h5 file
def save_model_as_h5(model, filename):
    # Save model state_dict in an h5 file using h5py
    with h5py.File(filename, 'w') as f:
        # Convert state_dict to numpy arrays
        for name, param in model.state_dict().items():
            f.create_dataset(name, data=param.cpu().numpy())

In [24]:
run_scenario('convnext',1,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

Number of training objects: 10000
Number of test objects: 2000
Experimento 1 - ConvNeXt (Sem Fine-Tuning)
----------------------------------------------------------------------------


[Epoch 1]: 100%|██████████████████| 625/625 [27:50<00:00,  2.67s/it, loss=0.667]


Train Loss: 0.043636 | Train Accuracy: 51.34% | Train F1-Score: 0.449627
Test Loss: 0.087110 | Test Accuracy: 51.70% | Test F1-Score: 0.446903




In [32]:
import torch
import torchvision.models as models

# Example: Initializing a model
mlmodel = models.convnext_tiny(pretrained=True)

In [33]:
torch.save(mlmodel.state_dict(), "convnext_scenario_1.h5")


In [34]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(df)

      Modelo  Experimento  Epoch  Train ACC  Train Loss  Train F1  Test ACC  \
11  convnext            1      1     0.5134    0.043636  0.449627     0.517   

    Test Loss   Test F1  
11    0.08711  0.446903  


In [ ]:
save_model_as_h5(mlmodel, "convnext_scenario_1_model_epoch_1.h5")

In [35]:
run_scenario('convnext',2,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)


Number of training objects: 10000
Number of test objects: 2000
Experiment 2 - ConvNeXt (With Fine-Tuning)
----------------------------------------------------------------------------


[Epoch 1]: 100%|██████████████| 625/625 [2:24:25<00:00, 13.86s/it, loss=0.00763]


Train Loss: 0.007432 | Train Accuracy: 94.92% | Train F1-Score: 0.949198
Test Loss: 0.010563 | Test Accuracy: 97.55% | Test F1-Score: 0.975485


----------------------------------------------------------------------------


[Epoch 2]: 100%|█████████████| 625/625 [2:29:58<00:00, 14.40s/it, loss=0.000356]


Train Loss: 0.001032 | Train Accuracy: 99.37% | Train F1-Score: 0.993700
Test Loss: 0.000715 | Test Accuracy: 99.80% | Test F1-Score: 0.998000


----------------------------------------------------------------------------


...

Train Loss: 0.000015 | Train Accuracy: 100.00% | Train F1-Score: 1.000000
Test Loss: 0.000356 | Test Accuracy: 99.90% | Test F1-Score: 0.999000


----------------------------------------------------------------------------


[Epoch 9]: 100%|████████████| 625/625 [13:47:20<00:00, 79.42s/it, loss=0.000122]


Train Loss: 0.000015 | Train Accuracy: 100.00% | Train F1-Score: 1.000000
Test Loss: 0.000356 | Test Accuracy: 99.90% | Test F1-Score: 0.999000


----------------------------------------------------------------------------


[Epoch 10]: 100%|███████████| 625/625 [11:30:13<00:00, 66.26s/it, loss=0.000147]


Train Loss: 0.000015 | Train Accuracy: 100.00% | Train F1-Score: 1.000000
Test Loss: 0.000356 | Test Accuracy: 99.90% | Test F1-Score: 0.999000




In [38]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

                      Epoch  Train ACC  Train Loss  Train F1  Test ACC  \
Modelo   Experimento                                                     
convnext 1              1.0    0.51340    0.043636  0.449627   0.51700   
         2              5.5    0.99426    0.000865  0.994260   0.99635   
         3              1.0    0.26760    0.048634  0.252744   0.49400   
         4              1.5    0.94705    0.007732  0.919874   0.98925   

                      Test Loss   Test F1  
Modelo   Experimento                       
convnext 1             0.087110  0.446903  
         2             0.001441  0.996349  
         3             0.088194  0.401016  
         4             0.003297  0.989248  


In [42]:
save_model_as_h5(mlmodel, "convnext_scenario_2_model_epoch_2.pth")

In [ ]:
run_scenario('convnext',3,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)


In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

In [ ]:
save_model_as_h5(mlmodel, "convnext_scenario_3_model_epoch_3.h5")

In [ ]:
run_scenario('convnext',4,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

In [ ]:
save_model_as_h5(mlmodel, "convnext_scenario_4_model_epoch_4.h5")

In [ ]:
run_scenario('convnext',5,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)


In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

In [ ]:
save_model_as_h5(mlmodel, "convnext_scenario_5_model_epoch_5.h5")

In [ ]:
import os
print(os.getcwd())


In [ ]:
#torch.save(mlmodel.state_dict(), "/Users/ananyapurkait/Study Files and Folders/Semester Study/Sem VII/CS435/Project/working/code/convnext_scenario_1.h5")